## Run All to load unity tennis with pretrained Model

In [1]:
import os
# os.environ['PATH'] = f"{os.environ['PATH']}:/home/student/.local/bin"
# os.environ['PATH'] = f"{os.environ['PATH']}:/opt/conda/lib/python3.10/site-packages"

os.environ['PATH'] = f"{os.environ['PATH']}:/home/vidy/.local/bin"
os.environ['PATH'] = f"{os.environ['PATH']}:/home/vidy/mambaforge/envs/py310/lib/python3.10/site-packages"


os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

In [2]:
!pip install .


Processing /home/vidy/RL_Tennis
  Preparing metadata (setup.py) ... done
  Created wheel for unityagents: filename=unityagents-0.4.0-py3-none-any.whl size=65045 sha256=cdd3fbb2493bde1b99edf857660d22520961b340f904d05b1fef073300623a1a
  Stored in directory: /tmp/pip-ephem-wheel-cache-mqr_b5fc/wheels/31/c0/e9/326c4b16106fcda9c5aa0cd49b733ed030f2ccbc054f5c0b34
Successfully built unityagents
  Attempting uninstall: unityagents
    Found existing installation: unityagents 0.4.0
    Uninstalling unityagents-0.4.0:
      Successfully uninstalled unityagents-0.4.0


In [3]:
from unityagents import UnityEnvironment
# Initialize Unity environment
env_path = "Tennis.x86_64"

env = UnityEnvironment(file_name=env_path, no_graphics=False)
brain_name = env.brain_names[0]

# Get state and action sizes from the environment
env_info = env.reset(train_mode=False)[brain_name]
state_size = env_info.vector_observations.shape[1]
action_size = 2   

Found path: /home/vidy/RL_Tennis/Tennis.x86_64
Mono path[0] = '/home/vidy/RL_Tennis/Tennis_Data/Managed'
Mono config path = '/home/vidy/RL_Tennis/Tennis_Data/MonoBleedingEdge/etc'
Preloaded 'ScreenSelector.so'
Preloaded 'libgrpc_csharp_ext.x64.so'
Unable to preload the following plugins:
	ScreenSelector.so
	libgrpc_csharp_ext.x86.so
Logging to /home/vidy/.config/unity3d/Unity Technologies/Unity Environment/Player.log


ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continu

In [4]:
import numpy as np
import torch
import os
from Agent import DDPGAgent

# Parameters
folder_path = 'Report'
GAMMA = 0.99
LEARNING_RATE = 1e-3
TAU = 0.3
DEVICE = 'cuda'
BUFFER_SIZE = 1_000_000
BATCH_SIZE = 128
n_episodes = 20

# Function to load pretrained models
def load_model(agent_1, agent_2, folder_path):
    agent_1.actor_local.load_state_dict(torch.load(os.path.join(folder_path, 'agent1_actor.pth')))
    agent_1.critic_local.load_state_dict(torch.load(os.path.join(folder_path, 'agent1_critic.pth')))
    agent_2.actor_local.load_state_dict(torch.load(os.path.join(folder_path, 'agent2_actor.pth')))
    agent_2.critic_local.load_state_dict(torch.load(os.path.join(folder_path, 'agent2_critic.pth')))

# Instantiate agents
agent_1 = DDPGAgent(state_size, action_size, random_seed=0, device=DEVICE,
                    learning_rate=LEARNING_RATE, tau=TAU, buffer_size=BUFFER_SIZE,
                    BATCH_SIZE=BATCH_SIZE, gamma=GAMMA)
agent_2 = DDPGAgent(state_size, action_size, random_seed=1, device=DEVICE,
                    learning_rate=LEARNING_RATE, tau=TAU, buffer_size=BUFFER_SIZE,
                    BATCH_SIZE=BATCH_SIZE, gamma=GAMMA)

# Load pretrained weights
load_model(agent_1, agent_2, folder_path)

# List to store episode scores
scores_list = []

# Run inference
for i in range(1, n_episodes + 1):
    env_info = env.reset(train_mode=False)[brain_name]  # Visual mode
    states = env_info.vector_observations
    scores = np.zeros(2)  # Scores for two agents

    while True:
        # Each agent acts without noise
        action_1 = agent_1.act(states[0], add_noise=False)
        action_2 = agent_2.act(states[1], add_noise=False)
        actions = np.concatenate((action_1, action_2), axis=0)

        # Step environment
        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations
        rewards = env_info.rewards
        dones = env_info.local_done

        scores += rewards
        states = next_states

        if np.any(dones):
            break

    max_score = np.max(scores)
    scores_list.append(max_score)
    print(f"Episode {i} - Scores: {scores} - Max: {max_score:.2f}")

# Close environment
env.close()

# Print average score
print(f"Average Score over {n_episodes} episodes: {np.mean(scores_list):.2f}")


Episode 1 - Scores: [ 0.   -0.01] - Max: 0.00
Episode 2 - Scores: [ 0.   -0.01] - Max: 0.00
Episode 3 - Scores: [0.1  0.09] - Max: 0.10
Episode 4 - Scores: [ 0.   -0.01] - Max: 0.00
Episode 5 - Scores: [0.1  0.09] - Max: 0.10
Episode 6 - Scores: [0.40000001 0.29      ] - Max: 0.40
Episode 7 - Scores: [ 0.   -0.01] - Max: 0.00
Episode 8 - Scores: [-0.01  0.  ] - Max: 0.00
Episode 9 - Scores: [0.1  0.09] - Max: 0.10
Episode 10 - Scores: [ 0.   -0.01] - Max: 0.00
Episode 11 - Scores: [ 0.   -0.01] - Max: 0.00
Episode 12 - Scores: [0.1  0.09] - Max: 0.10
Episode 13 - Scores: [ 0.   -0.01] - Max: 0.00
Episode 14 - Scores: [ 0.   -0.01] - Max: 0.00
Episode 15 - Scores: [0.1  0.09] - Max: 0.10
Episode 16 - Scores: [ 0.   -0.01] - Max: 0.00
Episode 17 - Scores: [ 0.1  -0.01] - Max: 0.10
Episode 18 - Scores: [ 0.1  -0.01] - Max: 0.10
Episode 19 - Scores: [0.3  0.19] - Max: 0.30
Episode 20 - Scores: [ 0.1  -0.01] - Max: 0.10
Average Score over 20 episodes: 0.08


We can see that 1 agent significantly perform better than the other.
This is because during training the goal is to achieve average score of 0.5